# Weather Data Visualization Notebook

Welcome to the Weather Data Visualization Notebook! This notebook is designed to help you fetch and visualize weather data for the past N (N>0) days.

## Setup Instructions

Before you start using this notebook, please follow these steps to set up the Conda environment:

1. **Create and Activate the Conda Environment**:
   
   Ensure you have Conda installed. Open a terminal and navigate to the root directory of the repository. Run the following command to create a new Conda environment based on the `agro_eye_notebooks.yaml` file located in the `conda` folder:

   ```bash
   conda env create -f conda/agro_eye_notebooks.yaml
   ```
### Install env variable

Put the following env variables in your `~/.env`:
- "AZURE_OPENAI_ENDPOINT",


In [8]:
# Import the necessary libraries
import requests
import json
from dotenv import load_dotenv
import os
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [9]:
load_dotenv(dotenv_path=os.path.expanduser("~/.env"))

True

In [12]:
default_env = {
    "API_WEATHER_KEY",
    "URL_OPEN-METEO"
}

In [13]:
for env in default_env:
    if env not in os.environ:
        raise Exception(f"Missing environment variable: {env}. Please add it to your ~/.env file")

Fetch historical Weather Forcast. The information has been fecthed from [here](https://open-meteo.com/en/docs#hourly=&daily=weather_code,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunrise,sunset,daylight_duration,uv_index_max,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration&start_date=2024-07-01&end_date=2024-07-09&time_mode=time_interval)

In [14]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [ ]:

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", "sunrise", "sunset", "daylight_duration", "uv_index_max", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
	"start_date": "2024-07-01",
	"end_date": "2024-07-09"
}
responses = openmeteo.weather_api(os.environ["URL_OPEN-METEO"], params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(4).ValuesAsNumpy()
daily_sunrise = daily.Variables(5).ValuesAsNumpy()
daily_sunset = daily.Variables(6).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(7).ValuesAsNumpy()
daily_uv_index_max = daily.Variables(8).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(9).ValuesAsNumpy()
daily_rain_sum = daily.Variables(10).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(11).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(12).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(13).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(14).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(15).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["uv_index_max"] = daily_uv_index_max
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)